<a href="https://colab.research.google.com/github/dvillasxUdg/ti_p3/blob/main/TI_1_Practica_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3

**Nombre:** Damian Alfonso Villaseñor Cisneros
**e-mail:** damian.villasenor3846@academicos.udg.mx

## MODULES

In [13]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

## CLASSES

In [3]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [4]:
###############################################################################################
# Turning angle
# This function calculates the turning angle between three consecutive positions
###############################################################################################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    vec_ab = #code here
    norm_ab = #code here

    vec_bc = #code here
    norm_bc = #code here

    dot_p = #code here

    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)

    # Angle orientation
    cross_p = #code here
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta



SyntaxError: ignored

## Actividad 1: Path Length - (BM1 vs BM2 vs CRW)

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [6]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [21]:
def bm_2d_from_dataframe(input_df, speed = 6):
    """
    Arguments:
      input_df: Dataframe 'x_pos' y 'y_pos'
    Returns:
      BM_2d_df: Brownian Motion DataFrame
    """
    n_steps = len(input_df)
    s_pos = [input_df['x_pos'].iloc[0], input_df['y_pos'].iloc[0]]

    velocity = Vec2d(speed, 0)

    BM_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps - 1):
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos.iloc[i] + velocity.x, 'y_pos': BM_2d_df.y_pos.iloc[i] + velocity.y}])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    return BM_2d_df

In [45]:
def correlated_random_walk_from_dataframe(input_df, speed=6, correlation=0.6):
    """
    Arguments:
      input_df: Dataframe 'x_pos' y 'y_pos'
      speed: Speed walker.
      correlation: CRW exponent.

    Returns:
      CRW_df: CRW DataFrame
    """
    n_steps = len(input_df)
    x_pos, y_pos = input_df['x_pos'].iloc[-1], input_df['y_pos'].iloc[-1]
    velocity = Vec2d(speed, 0)

    CRW_df = pd.DataFrame(columns=['x_pos', 'y_pos'])

    for _ in range(n_steps):
        turn_angle = wrapcauchy.rvs(correlation)
        velocity = velocity.rotated(turn_angle)
        x_pos += velocity.x
        y_pos += velocity.y
        CRW_df = CRW_df.append({'x_pos': x_pos, 'y_pos': y_pos}, ignore_index=True)

    return CRW_df

In [9]:
def euclidean_distance(point1, point2):
    return math.sqrt(sum((x - y) ** 2 for x, y in zip(point1, point2)))

In [48]:
# Compute path length
## start - Add your code here
# BM 3
dis_BM_3 = np.array([euclidean_distance(BM_2d_df_3.iloc[i-1], BM_2d_df_3.iloc[i]) for i in range(1, BM_2d_df_3.shape[0])])
pl_BM_3 = np.cumsum(dis_BM_3)
# BM 6
dis_BM_6 = np.array([euclidean_distance(BM_2d_df_6.iloc[i-1], BM_2d_df_6.iloc[i]) for i in range(1, BM_2d_df_6.shape[0])])
pl_BM_6 = np.cumsum(dis_BM_6)
# CRW
dis_CRW_9 = np.array([euclidean_distance(CRW_2d_df_9.iloc[i-1], CRW_2d_df_9.iloc[i]) for i in range(1, CRW_2d_df_9.shape[0])])
pl_CRW_9 = np.cumsum(dis_CRW_9)
## end - Add your code here

In [26]:
# Plotting
# Init figure
fig_path_length = go.Figure()

# First trace BM1
## start - Add your code here
fig_path_length.add_trace(go.Scatter(x=np.arange(len(pl_BM_3))+1,
                                     y=pl_BM_3,
                                     marker=dict(size=2),
                                     line=dict(width=2),
                                     mode='lines',
                                     name='Path length BM 3',
                                     showlegend=True))
## end - Add your code here


# Second trace BM2
## start - Add your code here
fig_path_length.add_trace(go.Scatter(x=np.arange(len(pl_BM_6))+1,
                                     y=pl_BM_6,
                                     marker=dict(size=2),
                                     line=dict(width=6),
                                     mode='lines',
                                     name='Path length BM 6',
                                     showlegend=True))
## end - Add your code here


# Third trace CRW
## start - Add your code here
fig_path_length.add_trace(go.Scatter(x=np.arange(len(pl_CRW_9))+1,
                                     y=pl_CRW_9,
                                     marker=dict(size=2),
                                     line=dict(width=2),
                                     mode='lines',
                                     name='Path length CRW 6',
                                     showlegend=True))
## end - Add your code here


fig_path_length.show()

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW)

* Generar una trayectoria tipo **BM** y una **CRW**.
* Implementar una función que calcule los valores de la curva de **mean squared displacement** de una trayectoria.
* Guardar metricas en Pandas Data Frame.
* Visualizar con **plotly**.

In [14]:
# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [15]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [18]:
# Empty MSD_BM
MSD_BM = np.empty(shape=(0))

# MSD for BM_2d_df_6
for tau in range(1, BM_2d_df_6.shape[0]):
    ## start - Add your code here
    deltas = BM_2d_df_6.iloc[tau:].values - BM_2d_df_6.iloc[:-tau].values
    sq_distances = np.sum(deltas**2, axis=1)
    msd_value = np.mean(sq_distances)
    MSD_BM = np.append(MSD_BM, msd_value)
    ## end - Add your code here

# Empty MSD_CRW
MSD_CRW = np.empty(shape=(0))

# MSD for CRW_2d_df_9
for tau in range(1, CRW_2d_df_9.shape[0]):
    ## start - Add your code here
    deltas = CRW_2d_df_9.iloc[tau:].values - CRW_2d_df_9.iloc[:-tau].values
    sq_distances = np.sum(deltas**2, axis=1)
    msd_value = np.mean(sq_distances)
    MSD_CRW = np.append(MSD_CRW, msd_value)
    ## end - Add your code here

# Save metrics to Dataframe
## start - Add your code here
msd_df = pd.DataFrame({'Tau': range(1, BM_2d_df_6.shape[0]), 'MSD_BM': MSD_BM, 'MSD_CRW': MSD_CRW})
## end - Add your code here

# Write to csv
## start - Add your code here
msd_df.to_csv('msd_results.csv', index=False)
## end - Add your code here

In [20]:
# Init figure
fig_path_length = go.Figure()

# first trace MSD_BM
## start - Add your code here
fig_path_length.add_trace(go.Scatter(x=msd_df['Tau'], y=msd_df['MSD_BM'], mode='lines', name='MSD BM 6'))
## end - Add your code here

# Second trace MSD_CRW
## start - Add your code here
fig_path_length.add_trace(go.Scatter(x=msd_df['Tau'], y=msd_df['MSD_CRW'], mode='lines', name='MSD CRW 6 c=9'))
## end - Add your code here

fig_path_length.show()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada)

* Generar **CRWs** con dos exponentes diferentes
* Guardar trayectorias en **Pandas** Data Frame
* Implementar una función que calcule los valores de **turning angle** de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [8]:
# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv('trajectories/crw_6_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [10]:
# aux to store turning angles
aux_ta_CRW_2d_df_6 = np.empty(shape=(0))


# Iterate over trajectory compute turning angles
for index, row in CRW_2d_df_6[1:-1].iterrows():
    ## start - Add your code here

    ## end - Add your code here

# aux to store turning angles
aux_ta_CRW_2d_df_9 = np.empty(shape=(0))


# Iterate over trajectory compute turning angles
for index, row in CRW_2d_df_9[1:-1].iterrows():
    ## start - Add your code here

    ## end - Add your code here


# Save to pandas DF
## start - Add your code here

## end - Add your code here


# Write to csv
## start - Add your code here

## end - Add your code here

In [12]:
# Check documentation
# https://plotly.com/python/histograms/

# PLot histogram
fig_met_df_3 = go.Figure()


# Histogram turning angle CRW_2d_df_6
## start - Add your code here

## end - Add your code here


# Histogram turning angle CRW_2d_df_9
## start - Add your code here

## end - Add your code here


# Add origin distributions
## start - Add your code here

## end - Add your code here


fig_met_df_3.show()

## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada)

* Implementar función que genere **Lévy Walks** (LW) utilizando pandas.
* Guardar trayectorias en Pandas Data Frame.
* Implementar una función que calcule los valores de **step lenght** de una trayectoria.
* Guardar trayectorias en **pandas** Data Frame.
* Obtener **Step-length** distribution.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con plotly.

In [ ]:
# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = pd.read_csv('trajectories/levy_6_1.csv')

# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv('trajectories/levy_6_7.csv')

In [ ]:
# aux to store turning angles
aux_ta_Levy_2d_df_1 = np.empty(shape=(0))
# aux to store step-lengths
aux_sl_Levy_2d_df_1 = np.empty(shape=(0))

## start - Add your code here

## end - Add your code here


# aux to store turning angles
aux_ta_Levy_2d_df_7 = np.empty(shape=(0))
# aux to store step-lengths
aux_sl_Levy_2d_df_7 = np.empty(shape=(0))

## start - Add your code here

## end - Add your code here

In [ ]:
# PLot histogram
fig_met_df_4 = go.Figure()

# Histogram step-length Levy_2d_df_1
## start - Add your code here

## end - Add your code here


# Histogram step-length Levy_2d_df_7
## start - Add your code here

## end - Add your code here


# Add origin distributions
## start - Add your code here

## end - Add your code here


fig_met_df_4.show()